In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('..')

# 内存保存历史 + 窗口记忆

In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个数学老师。"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
chain = prompt | model | StrOutputParser()

定义记忆管理器：

In [2]:
from langchain_chinese import MemoryManager
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory

window = ConversationBufferWindowMemory(
  return_messages=True, k=2, chat_memory = ChatMessageHistory()
)

memory = MemoryManager(shorterm_memory = window)

In [3]:
from langchain_chinese import WithMemoryBinding

withMemoryChain = WithMemoryBinding(
  chain,
  memory,
  input_messages_key="input",
  history_messages_key="history",
)

In [4]:
withMemoryChain.invoke(
  {"input": "三角函数什么意思？?"},
  config={"configurable": {"session_id": "abc123"}},
)

'三角函数是一类在三角形中定义的函数，它们以三角形的角度为自变量。这些函数包括正弦、余弦、正切、余切、正割和余割。在三角函数中，最常见的是正弦函数（sin）、余弦函数（cos）和正切函数（tan），它们分别表示了角的对应边与斜边之比。'

In [5]:
withMemoryChain.invoke(
  {"input": "为什么叫三角而不是四角?"},
  config={"configurable": {"session_id": "abc123"}},
)

'"三角"一词来自于这些函数最初是通过三角形的角度定义的事实。三角函数与三角形的角度相关，因此被称为三角函数。尽管这些函数最初是通过三角形定义的，但它们在数学中的应用远不止于此，涉及到许多其他领域，如物理学、工程学和计算机图形学等。'

In [7]:
withMemoryChain.invoke(
  {"input": "小学会学到吗?"},
  config={"configurable": {"session_id": "abc123"}},
)

'小学阶段通常不会深入学习三角函数，因为这需要一些基本的几何和代数知识。通常，三角函数在中学阶段的数学课程中会被介绍和学习，当学生已经具备了一定的数学基础时。'

In [8]:
memory.shorterm_messages("abc123")

[HumanMessage(content='为什么叫三角而不是四角?'),
 AIMessage(content='"三角"一词来自于这些函数最初是通过三角形的角度定义的事实。三角函数与三角形的角度相关，因此被称为三角函数。尽管这些函数最初是通过三角形定义的，但它们在数学中的应用远不止于此，涉及到许多其他领域，如物理学、工程学和计算机图形学等。'),
 HumanMessage(content='小学会学到吗?'),
 AIMessage(content='小学阶段通常不会深入学习三角函数，因为这需要一些基本的几何和代数知识。通常，三角函数在中学阶段的数学课程中会被介绍和学习，当学生已经具备了一定的数学基础时。')]

In [9]:
memory.longterm_messages("abc123")

[HumanMessage(content='三角函数什么意思？?'),
 AIMessage(content='三角函数是一类在三角形中定义的函数，它们以三角形的角度为自变量。这些函数包括正弦、余弦、正切、余切、正割和余割。在三角函数中，最常见的是正弦函数（sin）、余弦函数（cos）和正切函数（tan），它们分别表示了角的对应边与斜边之比。'),
 HumanMessage(content='为什么叫三角而不是四角?'),
 AIMessage(content='"三角"一词来自于这些函数最初是通过三角形的角度定义的事实。三角函数与三角形的角度相关，因此被称为三角函数。尽管这些函数最初是通过三角形定义的，但它们在数学中的应用远不止于此，涉及到许多其他领域，如物理学、工程学和计算机图形学等。'),
 HumanMessage(content='小学会学到吗?'),
 AIMessage(content='小学阶段通常不会深入学习三角函数，因为这需要一些基本的几何和代数知识。通常，三角函数在中学阶段的数学课程中会被介绍和学习，当学生已经具备了一定的数学基础时。')]

In [10]:
for chunk in withMemoryChain.stream(
  {"input": "我最初的问题是什么来着?"},
  config={"configurable": {"session_id": "abc123"}},
):
  print(chunk, end="|", flush=True)

|你|最|初|的|问题|是|关|于|为|什|么|称|呼|三|角|函数|而|不|是|四|角|函数|。||

# File持久化

In [1]:
from langchain_chinese.memory.history import LocalFileMessageHistory

memory = LocalFileMessageHistory(session_id='2024_04_28_161309_9499')

In [2]:
memory.session_id

'2024_04_28_161309_9499'

In [10]:
memory.add_user_message("请做我的助手吧")

In [3]:
memory.messages

[AIMessage(content='hi'), HumanMessage(content='请做我的助手吧')]

# redis持久化

```shell
pip install -U langchain-community redis
```

创建窗口管理器时要指定redis的工厂函数，其他与上面的例子相同。

In [ ]:
from langchain_chinese import MemoryManager
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain_community.chat_message_histories import RedisChatMessageHistory

window = ConversationBufferWindowMemory(
    return_messages=True, k=2, chat_memory = ChatMessageHistory()
)

memory = MemoryManager(
    lambda session_id: RedisChatMessageHistory(
        session_id, url="redis://localhost:6379"
    ),
    shorterm_memory = window
)